<a href="https://colab.research.google.com/github/carmea2025-dev/conversor-colab/blob/main/Conversor_BPA_Com_Botoes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
# @title Projeto BAHIA QUE PRODUZ E ALIMENTA - Plano de Ação { display-mode: "form" }

from IPython.display import display, HTML, clear_output
import pandas as pd
import io
import requests
from google.colab import files
import ipywidgets as widgets
from openpyxl import load_workbook
from openpyxl.styles import PatternFill, Alignment, Font
from openpyxl.drawing.image import Image
from datetime import datetime, date
import unicodedata
import re

# ============================================================
# TÍTULO VISÍVEL MESMO COM CÓDIGO OCULTO
# ============================================================
display(HTML("""
<h2 style="text-align:center; color:#2c3e50; font-family:'Trebuchet MS', sans-serif;">
📘 Projeto <b>BAHIA QUE PRODUZ E ALIMENTA</b> - Plano de Ação
</h2>
<hr style="border:1px solid #ccc;">
"""))

# ============================================================
# URLs DOS ARQUIVOS NO GITHUB
# ============================================================
url_mapeamento = "https://raw.githubusercontent.com/carmea2025-dev/conversor-colab/main/Mapeamento_Colunas.xlsx"
url_modelo = "https://raw.githubusercontent.com/carmea2025-dev/conversor-colab/main/PLANO%20DE%20AÇÃO%20-%20BPA%20-%20FINAL.xlsx"
url_bpa_img = "https://raw.githubusercontent.com/carmea2025-dev/conversor-colab/main/BPA.jpeg"
url_logos_img = "https://raw.githubusercontent.com/carmea2025-dev/conversor-colab/main/logos.jpeg"

# ============================================================
# FUNÇÃO PARA BAIXAR ARQUIVOS DO GITHUB
# ============================================================
def baixar_arquivo(url, nome_arquivo):
    r = requests.get(url)
    if r.status_code == 200:
        with open(nome_arquivo, "wb") as f:
            f.write(r.content)
        print(f"✅ {nome_arquivo} baixado com sucesso.")
    else:
        print(f"❌ Erro ao baixar {nome_arquivo}. Verifique o link do GitHub.")

baixar_arquivo(url_mapeamento, "Mapeamento_Colunas.xlsx")
baixar_arquivo(url_modelo, "PLANO_DE_ACAO_BPA_FINAL.xlsx")

# ============================================================
# FUNÇÃO PARA CARREGAR IMAGENS DO GITHUB
# ============================================================
def carregar_imagem_github(url, nome_temporario):
    r = requests.get(url)
    if r.status_code == 200:
        with open(nome_temporario, "wb") as f:
            f.write(r.content)
        return nome_temporario
    else:
        raise FileNotFoundError(f"Não foi possível carregar a imagem: {url}")

# ============================================================
# FUNÇÃO AUXILIAR PARA NORMALIZAÇÃO
# ============================================================
def normalize_text_full(s):
    if s is None:
        return ""
    s = str(s).strip().lower()
    s = ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')
    s = re.sub(r'[^a-z0-9\s]', '', s)
    s = re.sub(r'\s+', ' ', s).strip()
    return s

def parse_date(value):
    if pd.isnull(value):
        return None
    if isinstance(value, datetime):
        return value.date()
    if isinstance(value, date):
        return value
    try:
        return pd.to_datetime(str(value), dayfirst=True, errors='coerce').date()
    except:
        return None

# ============================================================
# FUNÇÃO DE CONVERSÃO
# ============================================================
def converter_arquivo(uploaded_file):
    try:
        mapeamento = pd.read_excel("Mapeamento_Colunas.xlsx", engine='openpyxl')
        modelo_df = pd.read_excel("PLANO_DE_ACAO_BPA_FINAL.xlsx", engine='openpyxl')

        nome_arquivo = list(uploaded_file.keys())[0]
        conteudo = uploaded_file[nome_arquivo]['content']

        if nome_arquivo.endswith(".xls"):
            df_bpa = pd.read_excel(io.BytesIO(conteudo), engine='xlrd')
        elif nome_arquivo.endswith(".xlsx"):
            df_bpa = pd.read_excel(io.BytesIO(conteudo), engine='openpyxl')
        else:
            print("⚠️ Formato de arquivo não suportado. Envie .xls ou .xlsx.")
            return

        col_map = dict(zip(mapeamento['Coluna_Origem'], mapeamento['Coluna_Destino']))
        df_convertido = df_bpa.rename(columns=col_map)

        wb = load_workbook("PLANO_DE_ACAO_BPA_FINAL.xlsx")
        ws = wb.active

        colunas_modelo = list(modelo_df.columns)
        df_convertido = df_convertido[[c for c in df_convertido.columns if c in colunas_modelo]]

        if "Ação" in df_convertido.columns:
            df_final = pd.merge(
                modelo_df,
                df_convertido,
                on="Ação",
                how="left",
                suffixes=("", "_NOVO")
            )
            for col in colunas_modelo:
                col_novo = col + "_NOVO"
                if col_novo in df_final.columns:
                    df_final[col] = df_final[col_novo].combine_first(df_final[col])
                    df_final.drop(columns=[col_novo], inplace=True, errors="ignore")
        else:
            df_final = modelo_df.copy()

        for col_idx, col_name in enumerate(colunas_modelo, start=1):
            if col_name not in df_final.columns:
                continue
            valores = df_final[col_name].tolist()
            for row_idx, valor in enumerate(valores, start=2):
                ws.cell(row=row_idx, column=col_idx, value=valor)

        # ============================================================
        # INSERIR CABEÇALHO, TÍTULO E IMAGENS
        # ============================================================
        ws.insert_rows(1)
        ws.merge_cells('A1:N1')
        cell = ws['A1']
        cell.value = 'PROJETO BAHIA QUE PRODUZ E ALIMENTA'
        cell.font = Font(name='Montserrat', size=40, bold=True)
        cell.alignment = Alignment(horizontal='center', vertical='center')
        ws.row_dimensions[1].height = 128.25

        # Carregar imagens
        img1_path = carregar_imagem_github(url_bpa_img, "BPA_temp.jpeg")
        img2_path = carregar_imagem_github(url_logos_img, "logos_temp.jpeg")

        img1 = Image(img1_path)
        img1.width, img1.height = 112, 112
        img2 = Image(img2_path)
        img2.width, img2.height = 700, 100

        img1.anchor = 'K1'
        img2.anchor = 'L1'

        ws.add_image(img1)
        ws.add_image(img2)

        # ============================================================
        # LINHA DE LEGENDA E RESUMO
        # ============================================================
        ws.insert_rows(2)
        ws.row_dimensions[2].height = 118
        ws.column_dimensions['L'].width = 39

        ws['L2'].value = (
            "Legenda - Progresso:\n"
            "- Verde - tarefa concluída;\n"
            "- Rosa - tarefa dentro do prazo;\n"
            "- Amarelo - tarefa com prazo para hoje;\n"
            "- Vermelho - tarefa com prazo expirado;\n"
            "- Azul - tarefa sem prazo planejado."
        )
        ws['L2'].alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)
        ws['M2'].value = "Data de atualização:"
        ws['M2'].alignment = Alignment(horizontal='right', vertical='center')
        ws['N2'].value = datetime.now().strftime('%d/%m/%Y')
        ws['N2'].alignment = Alignment(horizontal='center', vertical='center')

        # ============================================================
        # REGRAS DE CORES NA COLUNA "Progresso"
        # ============================================================
        rosa = PatternFill(start_color="FFC0CB", end_color="FFC0CB", fill_type="solid")
        amarelo = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")
        vermelho = PatternFill(start_color="FF0000", end_color="FF0000", fill_type="solid")
        verde = PatternFill(start_color="92D050", end_color="92D050", fill_type="solid")
        azul = PatternFill(start_color="ADD8E6", end_color="ADD8E6", fill_type="solid")
        centralizar = Alignment(horizontal="center", vertical="center")

        hoje = datetime.now().date()
        headers = [cell.value for cell in ws[3]]
        if "Progresso" in headers:
            idx_progresso = headers.index("Progresso") + 1
            idx_inicio = headers.index("Data de Início Planejada") + 1
            idx_conclusao = headers.index("Data de Conclusão Planejada") + 1

            for i in range(4, ws.max_row + 1):
                progresso_raw = ws.cell(i, idx_progresso).value
                progresso = normalize_text_full(progresso_raw)
                inicio_planejada = parse_date(ws.cell(i, idx_inicio).value)
                conclusao_planejada = parse_date(ws.cell(i, idx_conclusao).value)

                celula = ws.cell(i, idx_progresso)
                celula.alignment = centralizar

                if progresso == "nao iniciado":
                    if inicio_planejada is None:
                        celula.fill = azul
                    elif inicio_planejada > hoje:
                        celula.fill = rosa
                    elif inicio_planejada < hoje:
                        celula.fill = vermelho
                    else:
                        celula.fill = amarelo
                elif progresso == "em andamento":
                    if conclusao_planejada is None:
                        celula.fill = azul
                    elif conclusao_planejada > hoje:
                        celula.fill = rosa
                    elif conclusao_planejada < hoje:
                        celula.fill = vermelho
                    else:
                        celula.fill = amarelo
                elif progresso == "concluida":
                    celula.fill = verde
                else:
                    celula.fill = azul

        # ============================================================
        # RESUMO DE TAREFAS NA H2 e I2
        # ============================================================
        ws['H2'].value = "Resumo das tarefas:"
        ws['H2'].alignment = Alignment(horizontal='right', vertical='center', wrap_text=True)

        total_verde = total_vermelho = total_amarelo = total_rosa = total_azul = 0
        for i in range(4, 139):
            cell_fill = ws.cell(i, idx_progresso).fill
            if cell_fill.start_color.rgb is None:
                total_azul += 1
                continue
            cor = cell_fill.start_color.rgb[-6:]
            if cor == "92D050":
                total_verde += 1
            elif cor == "FF0000":
                total_vermelho += 1
            elif cor == "FFFF00":
                total_amarelo += 1
            elif cor == "FFC0CB":
                total_rosa += 1
            else:
                total_azul += 1

        resumo_texto = (
            f"{total_verde} tarefas concluídas\n"
            f"{total_vermelho} tarefas com atraso\n"
            f"{total_amarelo} tarefas com prazo para hoje - início ou conclusão\n"
            f"{total_rosa} tarefas dentro do prazo\n"
            f"{total_azul} tarefas sem prazo planejado"
        )
        ws['I2'].value = resumo_texto
        ws['I2'].alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)

        # ============================================================
        # RESUMO NA CÉLULA J2 (contagem azul e status)
        # ============================================================
        ws['J2'].value = ""
        ws['J2'].alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)

        total_concluidas = 0
        total_andamento = 0
        total_nao_iniciado = 0
        total_sem_prazo = 0

        for i in range(4, 139):
            valor = ws.cell(i, 8).value  # Coluna H
            cor_fill = ws.cell(i, 8).fill.start_color.rgb
            cor_hex = cor_fill[-6:] if cor_fill is not None else ""

            if valor is not None:
                texto = normalize_text_full(valor)
                if texto == "concluida":
                    total_concluidas += 1
                elif texto == "em andamento":
                    total_andamento += 1
                elif texto == "nao iniciado":
                    total_nao_iniciado += 1

            # Células azuis = tarefas sem prazo
            if cor_hex.upper() == "ADD8E6":
                total_sem_prazo += 1

        resumo_j2 = (
            f"{total_concluidas} tarefas concluídas\n"
            f"{total_andamento} tarefas em andamento\n"
            f"{total_nao_iniciado} tarefas não iniciadas\n"
            f"{total_sem_prazo} tarefas sem prazo planejado"
        )
        ws['J2'].value = resumo_j2

        # ============================================================
        # SALVAR ARQUIVO
        # ============================================================
        nome_saida = "PLANO DE AÇÃO - BPA - CONVERTIDO.xlsx"
        wb.save(nome_saida)
        print("✅ Conversão concluída com sucesso! Cabeçalho, legenda, cores e resumo de tarefas atualizados.")
        files.download(nome_saida)

    except Exception as e:
        print(f"❌ Erro durante a conversão: {e}")

# ============================================================
# INTERFACE NO COLAB
# ============================================================
def interface():
    upload_button = widgets.FileUpload(accept=".xls,.xlsx", multiple=False)
    convert_button = widgets.Button(description="Converter e Baixar", button_style="success")

    def ao_clicar(_):
        clear_output(wait=True)
        display(HTML("""
        <h2 style="text-align:center; color:#2c3e50; font-family:'Trebuchet MS', sans-serif;">
        📘 Projeto <b>BAHIA QUE PRODUZ E ALIMENTA</b> - Plano de Ação
        </h2>
        <hr style="border:1px solid #ccc;">
        """))
        display(upload_button, convert_button)
        if upload_button.value:
            converter_arquivo(upload_button.value)
        else:
            print("⚠️ Envie um arquivo BPA primeiro.")

    convert_button.on_click(ao_clicar)
    display(upload_button, convert_button)

interface()


FileUpload(value={'PLANO DE AÇÃO - BPA.xls': {'metadata': {'name': 'PLANO DE AÇÃO - BPA.xls', 'type': 'applica…

Button(button_style='success', description='Converter e Baixar', style=ButtonStyle())

✅ Conversão concluída com sucesso! Cabeçalho, legenda, cores e resumo de tarefas atualizados.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>